# Week 04 Assignment weather data

Welcome to week four of this course programming 1. Analyzing time related data such as estimating seasonal effect, or year effect might be a challenge. How to filter the essential information from the noise? How to apply signal analysis with noisy data. How to make compact useful visualizations? Python has several constructs to handle date time related data. The relevant classes for making plots are Locators and Formatters. Locators determine where the ticks are, and formatters control the formatting of tick labels. The relevant class for date time data is the pandas datetime data type, which has methods like resample and several possibilities to display data (frequencies). As a study case we will work with weather data. If you have data that fits the learning goals, you can bring your own data.

Keywords: signal processing, smoothing, resample, formatters and locators, datetime object

More to read: 

- https://fennaf.gitbook.io/bfvm19prog1/
- https://machinelearningmastery.com/time-series-data-visualization-with-python/
- https://towardsdatascience.com/how-to-plot-time-series-86b5358197d6
- In the https://pandas.pydata.org/docs/reference/offset_frequency.html you can find more about frequencies and in the documentation
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html you can read all the methods of this datetime object.

Learning objectives

- load, inspect and clean a dataset
- reshape dataframes to group data in a certain frequency
- apply smoothing technologies
- Create useful visualisation with timeseries data
- Maintain development environment 
- Apply coding standards and FAIR principles


## Assignment

You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below. 

<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. widgets selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


NB if you want to make a plot with more actual data you can download data from https://openweathermap.org/api 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from a station. Station 270 is in the neighborhood of Groningen. For our plot we only need the the mean, minimum and maximum temperature. Of course you are welcome to select other data if you think it might be useful for your visualization. The data should look something like this:


---

In [1007]:
# import packages
import pandas as pd
import numpy as np 
from datetime import date, datetime
from bokeh.plotting import figure, show, output_file, show
from bokeh.layouts import gridplot, row, column
from bokeh.models import CheckboxGroup, CustomJS, Panel, Tabs, Band, ColumnDataSource, Slider, CustomJS, Range1d
from bokeh.io import output_notebook
output_notebook()

# Load the data
df = pd.read_csv("../data/KNMI_20181231.txt.tsv", sep= ',', skiprows= 63, low_memory=False, skipinitialspace=True )
df = pd.DataFrame(df) 
df = df.drop(index= 0)
df.rename(columns={'# STN': 'station', 'YYYYMMDD': 'DATE', 'TG':'Tmean', 'TN':'Tmin', 'TX':'Tmax', 'SQ':'Sun(H)', 'DR':'Rain(H)', 'RH':'rain(mm)'}, inplace=True)
df = df.astype({'DATE': 'int64'})
df = df.loc[(df.station == '270')]

print(f"Total number of 270 stations ", len(df.station))


# Column information
# YYYYMMDD = Date (YYYY=year MM=month DD=day);
# TG       = Temperature average  (in 0.1 degrees Celsius);
# TN       = Minimum Temperature (in 0.1 degrees Celsius);
# TX       = Maximum Temperature (in 0.1 degrees Celsius);
# SQ       = sun shine time (in 0.1 hour);
# DR       = rain fall time (in 0.1 hour);
# RH       = sun of rain fall (in 0.1 mm);

# test code
# Data for location information (not needed for assigment), also data selecting is not working. 
# # Load in the data
# KNMI = pd.read_csv("../data/KNMI_20181231.txt.tsv", sep='/t', )
# print(KNMI)

# dfone = pd.read_csv("../data/KNMI_20181231.txt.tsv", sep= '/t', skiprows= 3, nrows=50)
# dfone = pd.DataFrame(dfone)
# dfone


Loading BokehJS ...

Total number of 270 stations  6940


<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [1008]:
# renomve unwanted information and empty columns
# Replace empty cells with NaN
df = df.replace(r'^\s*$', np.NaN, regex=True)
# df.head()

# Change data type of the data into datetime format
# pd.to_datetime(df['DATE'].astype(str), format='%Y%m%d')
# Test how the lambda version worked
df[['DATE']] = (
    df[['DATE']]
    .applymap(str)
    .applymap(lambda s: "{}-{}-{}".format(s[4:6], s[6:], s[:4]))
)

# change centidegrees into degrees
#Say you want to divide every number in your_list by some number
df[['Tmean', 'Tmin', 'Tmax']]= df[['Tmean', 'Tmin', 'Tmax']].div(10)

#renomve leap years days
#remove month == 2 & day == 29
df[~df.DATE.astype(str).str.endswith('02-29')]
# check for leap year data in dable
leap = df.loc[(df.station == '02-29')]
print(f"Leap days ", len(leap))
df = df.astype({'DATE': 'datetime64[ns]', 'station':'int64'})
Temp_data = df[["station", "DATE", "Tmean", "Tmin", "Tmax"]]

Temp_data.head()


Leap days  0


,station,DATE,Tmean,Tmin,Tmax
97642,270,2000-01-01,4.2,-0.4,7.9
97643,270,2000-01-02,5.5,3.3,7.4
97644,270,2000-01-03,7.4,4.9,8.9
97645,270,2000-01-04,4.6,2.2,7.5
97646,270,2000-01-05,4.1,1.4,5.6


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [1009]:
#Test your outcome
Temp_data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 97642 to 104581
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  6940 non-null   int64         
 1   DATE     6940 non-null   datetime64[ns]
 2   Tmean    6940 non-null   float64       
 3   Tmin     6940 non-null   float64       
 4   Tmax     6940 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 325.3 KB


### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [1010]:

# Selecting 2018
data2018 = (Temp_data['DATE'] > '2017-12-31') # 2018 data
data2018 = Temp_data.loc[data2018]

# Selecting everything before 2018    
before2018 = (Temp_data['DATE'] <= '2017-12-31') # before 2018 data
before2018 = Temp_data.loc[before2018]

def month_day(df_multipleyears):
      temp_min = before2018.groupby([(before2018.DATE.dt.month),(before2018.DATE.dt.day)]).min()
      temp_min.index.names = ['Month', 'Day']
      print(temp_min)
      
      temp_max = before2018.groupby([(before2018.DATE.dt.month),(before2018.DATE.dt.day)]).max()
      temp_max.index.names = ['Month', 'Day']
      print(temp_max)
    
month_day(before2018)

      
temp_min = before2018.groupby([(before2018.DATE.dt.month),(before2018.DATE.dt.day)]).min()
temp_min.index.names = ['Month', 'Day']
      
temp_max = before2018.groupby([(before2018.DATE.dt.month),(before2018.DATE.dt.day)]).max()
temp_max.index.names = ['Month', 'Day']

           station       DATE  Tmean  Tmin  Tmax
Month Day                                       
1     1        270 2000-01-01   -1.9  -5.8   0.4
      2        270 2000-01-02   -3.4  -7.5  -1.4
      3        270 2000-01-03   -4.1 -12.6  -1.2
      4        270 2000-01-04   -3.8  -6.7  -0.2
      5        270 2000-01-05   -2.3  -6.2  -1.2
...            ...        ...    ...   ...   ...
12    27       270 2000-12-27   -2.0  -6.0  -0.3
      28       270 2000-12-28   -2.9  -7.4   0.3
      29       270 2000-12-29   -4.0  -7.3  -2.6
      30       270 2000-12-30   -6.0 -10.2  -2.4
      31       270 2000-12-31   -4.5 -10.6   0.0

[366 rows x 5 columns]
           station       DATE  Tmean  Tmin  Tmax
Month Day                                       
1     1        270 2017-01-01    9.8   8.2  11.1
      2        270 2017-01-02    7.8   5.5  10.2
      3        270 2017-01-03    9.0   7.1  10.7
      4        270 2017-01-04    8.9   7.6   9.8
      5        270 2017-01-05    8.4   6.1   

In [1011]:
# testing code
# selecting specifi dates 
# Between dates
# mask = (Temp_data['DATE'] > 'End Date') & (Temp_data['DATE'] <= 'Start Date')
# print(Temp_data.loc[mask])
# data2018 = (Temp_data['DATE'] > '2017-12-31') # 2018 data
# befrore2018 = (Temp_data['DATE'] <= '2017-12-31') # before 2018 data

# Temp_data[["day", "month", "year"]] = Temp_data["DATE"].str.split("-", expand = True)

# temp_max = before2018.groupby([(before2018.DATE.dt.month),(before2018.DATE.dt.day)]).max()
# temp_max.index.names = ['Month', 'Day']
# temp_max

# # test code
# # Reset our index so datetime_utc becomes a column
# data2018.reset_index(inplace=True)

# # Create new columns
# data2018['day'] = data2018['DATE'].dt.day
# data2018['month'] = data2018['DATE'].dt.month
# data2018['year'] = data2018['DATE'].dt.year

# data2018 = data2018[["DATE", "day", "month", "year", "Tmean", "Tmin", "Tmax"]]
# data2018


In [1012]:
#Test your code
# def test_reformed(df)
#     #
#     df = df[(df.index.year > 2000) & (df.index.year < 2018)]
#     month_day(df)

# test_reformed(data2018)

print(temp_min, temp_max)

           station       DATE  Tmean  Tmin  Tmax
Month Day                                       
1     1        270 2000-01-01   -1.9  -5.8   0.4
      2        270 2000-01-02   -3.4  -7.5  -1.4
      3        270 2000-01-03   -4.1 -12.6  -1.2
      4        270 2000-01-04   -3.8  -6.7  -0.2
      5        270 2000-01-05   -2.3  -6.2  -1.2
...            ...        ...    ...   ...   ...
12    27       270 2000-12-27   -2.0  -6.0  -0.3
      28       270 2000-12-28   -2.9  -7.4   0.3
      29       270 2000-12-29   -4.0  -7.3  -2.6
      30       270 2000-12-30   -6.0 -10.2  -2.4
      31       270 2000-12-31   -4.5 -10.6   0.0

[366 rows x 5 columns]            station       DATE  Tmean  Tmin  Tmax
Month Day                                       
1     1        270 2017-01-01    9.8   8.2  11.1
      2        270 2017-01-02    7.8   5.5  10.2
      3        270 2017-01-03    9.0   7.1  10.7
      4        270 2017-01-04    8.9   7.6   9.8
      5        270 2017-01-05    8.4   6.1   

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm. Ask the signal analysis teacher if you want some advice.


In [1013]:
#your code here
import numpy as np
from scipy.signal import savgol_filter

x = data2018.DATE
data2018['Tmean_int'] = savgol_filter(data2018['Tmean'], 51, 3)
data2018['Tmin_int'] = savgol_filter(data2018['Tmin'], 51, 3)
data2018['Tmax_int'] = savgol_filter(data2018['Tmax'], 51, 3)

data2018.head()

#your motivation here
# Savitzky–Golay filter is a digital filter that uses data points for smoothing the graph. 
# It uses the method of least squares that creates a small window and applies a polynomial on the data of that window, 
# and then uses that polynomial for assuming the center point of the particular window. Next, the window is shifted by one data point, 
# and the process is iterated until all the neighbors are relatively adjusted with each other.
# Removing diffrence between peak values, while still following the same pattern

p1 = figure(title="Temperture distrubution of 2018", x_axis_label='Date', y_axis_label='Temperature(celsius)', x_axis_type='datetime')
p1.line(data2018["DATE"], data2018["Tmean"], legend_label="Tmean", line_width=2, color = "blue")
p1.line(data2018["DATE"], data2018["Tmin"], legend_label="Tmin", line_width=2, color = "red")
p1.line(data2018["DATE"], data2018["Tmax"], legend_label="Tmax", line_width=2, color = "green")
p1.legend.click_policy = "hide"
# show(p1)

p2 = figure(title="smmothed Temperture distrubution of 2018", x_axis_label='Date', y_axis_label='Temperature(in celsius)', x_axis_type='datetime')
p2.line(data2018["DATE"], data2018["Tmean_int"], legend_label="Tmean", line_width=2, color = "blue")
p2.line(data2018["DATE"], data2018["Tmin_int"], legend_label="Tmin", line_width=2, color = "red")
p2.line(data2018["DATE"], data2018["Tmax_int"], legend_label="Tmax", line_width=2, color = "green")
p2.legend.click_policy = "hide"
# show(p2)

# p3 = figure(title="Temperture distrubution of 2000-2018", x_axis_label='Date', y_axis_label='Temperature(in celsius)', x_axis_type='datetime')
# p3.line(Temp_data["DATE"], Temp_data["Tmean"], legend_label="Tmean", line_width=2, color = "blue")
# p3.line(data2018["DATE"], Temp_data["Tmin"], legend_label="Tmin", line_width=2, color = "red")
# p3.line(Temp_data["DATE"], Temp_data["Tmax"], legend_label="Tmax", line_width=2, color = "green")
# p3.legend.click_policy = "hide"
# # show(p3)


grid = gridplot([p1, p2], ncols=3)
show(grid)


---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

In [1014]:
from bokeh.models import Range1d

# smooth ultimate min and max

# temp_min['Tmean_int'] = savgol_filter(temp_min['Tmean'], 51, 3)
# temp_min['Tmin_int'] = savgol_filter(temp_min['Tmin'], 51, 3)
# temp_min['Tmax_int'] = savgol_filter(temp_min['Tmax'], 51, 3)

# temp_max['Tmean_int'] = savgol_filter(temp_min['Tmean'], 51, 3)
# temp_max['Tmin_int'] = savgol_filter(temp_min['Tmin'], 51, 3)
# temp_max['Tmax_int'] = savgol_filter(temp_min['Tmax'], 51, 3)

# temp_min = temp_min['Tmin']
# temp_max = temp_max['Tmax']
# # print(temp_min, temp_max, data2018)
# data2018.merge(data2018, temp_min)
# data2018.merge(data2018, temp_max)


temp_min.reset_index(drop=True, inplace=True)
temp_max.reset_index(drop=True, inplace=True)
data2018.reset_index(drop=True, inplace=True)
# print(temp_min, temp_max)

# I could not figure out how to merge the indexed column with a non group indexed column, 
# for this reason the plot band only contains of 2018 data and not the all years range

# Data
x = data2018.DATE
y = data2018.Tmean_int
yrange = Range1d(-5,30)
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

df = pd.DataFrame(data=dict(x=x, y=y)).sort_values(by="x")

df = pd.concat([df, df2], axis=1)
df['lower'] = data2018.Tmin_int
df['upper'] = data2018.Tmax_int

p = figure(title="Temperture distrubution of 2018", y_range = yrange, x_axis_label='Date', y_axis_label='Temperature(in celsius)', x_axis_type='datetime', tools = TOOLS)
p.line(data2018["DATE"], data2018["Tmean_int"], legend_label="Tmean 2018", line_width=2, color = "blue")
p.line(data2018["DATE"], data2018["Tmin_int"], legend_label="Tmin 2018", line_width=2, color = "red")
p.line(data2018["DATE"], data2018["Tmax_int"], legend_label="Tmax 2018", line_width=2, color = "green")
source = ColumnDataSource(df.reset_index())

band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
            fill_alpha=1.0, line_width=1, line_color='red')
p.add_layout(band)



p.legend.click_policy = "hide"

show(p)


---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Or maybe a widget were you choose a different station. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather

In [1015]:


# not completed, could not figure out how to connect the slider to the years. 
# (i'll assume i needed to group my dataframe by year for that and then connect each year to the output)
# will work on this at a later stage further

In [1016]:
x = Temp_data.DATE
y = Temp_data.Tmean

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(title="Temperture distrubution", y_range = yrange, x_axis_label='Date', y_axis_label='Temperature(in celsius)', x_axis_type='datetime')

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp_slider = Slider(start=2000, end=2018, value=1, step=.1, title="year")

callback = CustomJS(args=dict(source=source, amp=amp_slider),
                    code="""
    const data = source.data;
    const A = amp.value;
    const k = freq.value;
    const phi = phase.value;
    const B = offset.value;
    const x = data['x']
    const y = data['y']
    for (let i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.change.emit();
""")

amp_slider.js_on_change('value', callback)


layout = row(
    plot,
    column(amp_slider),
)

show(layout)